## Example Experiment ##
### Dataset: Labeled Faces in the Wild ###
### Experiment: Two party training for gender classification ###

In [1]:
import simulator.server as server
import simulator.workerclass as worker
import simulator.workerhandler as wh
import topology_utils as tu


import pandas as pd
import numpy as np
import torchvision.models as models
import copy

## Divide Data between workers ##

In [2]:
data_path = '../data/lfw/data/'

attributes_df = pd.read_csv(data_path+'lfw_attributes.txt')
print(len(attributes_df))
# add column to indicate split
attributes_df['target'] = 0
# allocate half the people to the target
names = attributes_df['person'].drop_duplicates()
target_worker_names = names.sample(frac=1)[:int(len(names)/2)]
target_worker_names = target_worker_names.reset_index(drop=True)

# populate target field
for index, row in attributes_df.iterrows():
    if row['person'] in target_worker_names.values:
        attributes_df['target'][index] = 1

13143


/home/sattvik/envs/pytorch_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Define Model ##

In [3]:
def define_model():
    return models.resnet18()

## Declare genuine worker ##

In [13]:
class target_worker(worker.base_workerclass):
    def __init__(self, attributes_df, model):
        super().__init__(False)
        self.worker_attributes_df = attributes_df[attributes_df['target']==1]
        print("initializing malicious worker node with ",len(self.worker_attributes_df)," data points")
        self.model = model
    
    def set_param(self, w):
        self.model.load_state_dict(w)

## Declare malicious worker ##

In [14]:
class malicious_worker(worker.base_workerclass):
    def __init__(self, attributes_df, model):
        super().__init__(True)
        self.worker_attributes_df = attributes_df[attributes_df['target']==0]
        print("initializing malicious worker node with ",len(self.worker_attributes_df)," data points")
        self.model = model
    
    def set_param(self, w):
        self.model.load_state_dict(w)

## Initialize components of our simulations ##

In [15]:
server1 = server.server()
workers = wh.workerhandler([target_worker(attributes_df,define_model()),malicious_worker(attributes_df,define_model())])
tm = tu.topology_manager()

initializing malicious worker node with  6421  data points
tensor([[[[-2.8879e-03, -3.3731e-02,  4.1794e-02,  ...,  1.4103e-02,
           -4.9707e-03, -3.4668e-02],
          [-6.8094e-03,  6.6791e-03, -2.4282e-02,  ..., -3.7501e-02,
            1.3364e-02, -1.7085e-02],
          [ 1.6095e-02,  5.0069e-02,  1.1229e-03,  ...,  6.6400e-03,
           -1.5079e-02,  2.2483e-02],
          ...,
          [-1.3252e-03,  2.1004e-02,  1.9122e-02,  ..., -1.9855e-02,
            2.4862e-02, -5.9752e-03],
          [-1.0410e-02, -3.9255e-03, -1.3445e-02,  ...,  2.2523e-02,
           -1.8434e-02, -3.6000e-02],
          [-5.0775e-02,  1.7462e-02,  2.4896e-02,  ..., -1.5797e-02,
            6.1210e-03,  1.7193e-02]],

         [[ 3.5069e-02,  1.4763e-04,  2.8828e-02,  ..., -2.5449e-02,
           -3.0803e-02,  3.4851e-02],
          [-1.0454e-02, -3.6007e-02, -9.0743e-03,  ..., -1.6476e-02,
           -5.3158e-02,  1.7869e-02],
          [-4.7456e-03,  1.5952e-02, -1.6303e-02,  ..., -4.8882e-03,

## Define network topology ##

In [ ]:
tm.connect_star(server1, workers.get_all_workers())

In [ ]:
plot = tm.plot_topology()